In [184]:
import sys
import os
sys.path.append(os.path.abspath('.'))

In [ ]:
# load dataset
from pages.A_Explore_Preprocess_Dataset import load_dataset
filepath = "housing_dataset.csv"
df = load_dataset(filepath)
df.head()

2025-05-10 13:44:10.719 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 13:44:10.720 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
# print basic statistics about dataset - number of features, distribution of target variable
print(f"Number of rows: {df.shape[0]}")
print(f"Number of features: {df.shape[1]}")

numeric_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

print(f"Number of numeric features: {len(numeric_cols)}")
print(f"Number of categorical features: {len(categorical_cols)}")

num_rows_with_missing = df.isnull().any(axis=1).sum()
print(f"% of rows with missing values: {num_rows_with_missing/df.shape[0] * 100:.2f}%")

print("Counts of 0 and 1:", df['TARGET'].value_counts())

Number of rows: 307511
Number of features: 122
Number of numeric features: 106
Number of categorical features: 16
% of rows with missing values: 97.20%
Counts of 0 and 1: TARGET
0    282686
1     24825
Name: count, dtype: int64


In [171]:
# impute missing
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# mean for numeric features
for col in numeric_cols:
    if df[col].isnull().any():
        df[col].fillna(df[col].mean(), inplace=True)

# mode for categorical features
for col in categorical_cols:
    if df[col].isnull().any():
        df[col].fillna(df[col].mode()[0], inplace=True)

# assert no missing values
num_rows_with_missing = df.isnull().any(axis=1).sum()
print(f"% of rows with missing values: {num_rows_with_missing/df.shape[0] * 100:.2f}%")

/var/folders/6m/b61mk3xj41d29b66x2p3pggw0000gn/T/ipykernel_94682/1569036470.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
/var/folders/6m/b61mk3xj41d29b66x2p3pggw0000gn/T/ipykernel_94682/1569036470.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

% of rows with missing values: 0.00%


In [172]:
# for each categorical column, print how many unique values it has
for col in categorical_cols:
    print(f"{col}: {df[col].nunique()} unique values")
    print(df[col].value_counts())
    print()

# organization type has 58 unique values, so we will group those with counts < 2000 into "other"
org_counts = df['ORGANIZATION_TYPE'].value_counts()
threshold = 500

rare_orgs = org_counts[org_counts < threshold].index
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace(rare_orgs, 'Other')

NAME_CONTRACT_TYPE: 2 unique values
NAME_CONTRACT_TYPE
Cash loans         278232
Revolving loans     29279
Name: count, dtype: int64

CODE_GENDER: 3 unique values
CODE_GENDER
F      202448
M      105059
XNA         4
Name: count, dtype: int64

FLAG_OWN_CAR: 2 unique values
FLAG_OWN_CAR
N    202924
Y    104587
Name: count, dtype: int64

FLAG_OWN_REALTY: 2 unique values
FLAG_OWN_REALTY
Y    213312
N     94199
Name: count, dtype: int64

NAME_TYPE_SUITE: 7 unique values
NAME_TYPE_SUITE
Unaccompanied      249818
Family              40149
Spouse, partner     11370
Children             3267
Other_B              1770
Other_A               866
Group of people       271
Name: count, dtype: int64

NAME_INCOME_TYPE: 8 unique values
NAME_INCOME_TYPE
Working                 158774
Commercial associate     71617
Pensioner                55362
State servant            21703
Unemployed                  22
Student                     18
Businessman                 10
Maternity leave              5
Name:

In [173]:
# convert categorical features to numerical
from pages.A_Explore_Preprocess_Dataset import one_hot_encode_feature 

df = one_hot_encode_feature(df, categorical_cols)

print(len(df.columns))

2025-05-10 13:44:17.056 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 13:44:17.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 13:44:17.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 13:44:17.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 13:44:17.058 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 13:44:17.058 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


229


In [174]:
# normalize all features
features = [col for col in df.columns if col != 'TARGET' and col != 'SK_ID_CURR']
X = df[features]
y = df['TARGET']

for f in features:
    X[f] = (X[f] - X[f].min()) / (X[f].max() - X[f].min())  
X.head()

/var/folders/6m/b61mk3xj41d29b66x2p3pggw0000gn/T/ipykernel_94682/3634401956.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[f] = (X[f] - X[f].min()) / (X[f].max() - X[f].min())
/var/folders/6m/b61mk3xj41d29b66x2p3pggw0000gn/T/ipykernel_94682/3634401956.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[f] = (X[f] - X[f].min()) / (X[f].max() - X[f].min())
/var/folders/6m/b61mk3xj41d29b66x2p3pggw0000gn/T/ipykernel_94682/3634401956.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy o

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0.0,0.001512,0.090287,0.090032,0.077441,0.256321,0.888839,0.045086,0.852140,0.705433,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.002089,0.311736,0.132924,0.271605,0.045016,0.477114,0.043648,0.951929,0.959566,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.000358,0.022472,0.020025,0.023569,0.134897,0.348534,0.046161,0.827335,0.648326,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.000935,0.066837,0.109477,0.063973,0.107023,0.350846,0.038817,0.601451,0.661387,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.000819,0.116854,0.078975,0.117845,0.392880,0.298591,0.038820,0.825268,0.519522,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


We have filled in missing data, covnerted categorical to numeric, and normalized features. We started with 122 features, now we have 227. Next, we will reduce the number of features, filtering out those with low correlation with target variable.

In [215]:
# measure collinearity and correlation with target variable to reduce number of features
import numpy as np

# correlation with target check
cor_with_target = X.apply(lambda col: np.corrcoef(col, y)[0, 1])
cor_threshold = 0.02  # TODO: try different values

strong_features = cor_with_target[abs(cor_with_target) >= cor_threshold].index.tolist()
X_filtered = X[strong_features]

# collinearity check
corr_matrix = X_filtered.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
collinearity_threshold = 0.9 # TODO: try different values
to_drop = [column for column in upper.columns if any(upper[column] > collinearity_threshold)]

# drop
X_final = X_filtered.drop(columns=to_drop)
print("Original # features:", df.shape[1] - 1)
print("After correlation filtering:", len(strong_features))
print("After collinearity filtering:", X_final.shape[1])

Original # features: 228
After correlation filtering: 58
After collinearity filtering: 43


In [211]:
print(X_final.columns)

Index(['DAYS_BIRTH', 'DAYS_ID_PUBLISH', 'REGION_RATING_CLIENT',
       'REG_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3', 'DAYS_LAST_PHONE_CHANGE', 'CODE_GENDER_F',
       'NAME_INCOME_TYPE_Working', 'NAME_EDUCATION_TYPE_Higher education'],
      dtype='object')


Now we can begin modeling!

In [221]:
from sklearn.model_selection import train_test_split
from pages.B_Train_Model import LogisticRegression

X_train, X_val, y_train, y_val = train_test_split(
    X_final.values, y.values, test_size=0.3, random_state=42
)

# train logistic regression
model = LogisticRegression(learning_rate=0.001, num_iterations=500) # TODO: try diff learning rate, iterations
model.fit(X_train, y_train)

In [206]:
from pages.C_Test_Model import compute_accuracy, compute_precison_recall

def test_model(model, X_train, y_train, X_val, y_val):
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)

    train_acc = compute_accuracy(y_pred_train, y_train)
    train_precision, train_recall = compute_precison_recall(y_pred_train, y_train)

    val_acc = compute_accuracy(y_pred_val, y_val)
    val_precision, val_recall = compute_precison_recall(y_pred_val, y_val)

    print(f"Train accuracy: {train_acc * 100:.2f}%, Train precision: {train_precision * 100:.2f}%, Train recall: {train_recall * 100:.2f}%")
    print(f"Validation accuracy: {val_acc * 100:.2f}%, Validation precision: {val_precision * 100:.2f}%, Validation recall: {val_recall * 100:.2f}%")

In [222]:
test_model(model, X_train, y_train, X_val, y_val)

2025-05-10 14:16:09.409 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 14:16:09.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 14:16:09.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 14:16:09.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 14:16:09.418 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 14:16:09.418 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 14:16:09.418 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 14:16:09.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Train accuracy: 0.00%, Train precision: 0.00%, Train recall: 0.00%
Validation accuracy: 0.00%, Validation precision: 0.00%, Validation recall: 0.00%


In [27]:
!pip install -q matplotlib

In [27]:
# plot loss curves
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(model.cost_history, label='Linear Regression', linewidth=2)
plt.plot(lasso_model.cost_history, label='Lasso Regression', linestyle='--')
plt.plot(ridge_model.cost_history, label='Ridge Regression', linestyle=':')
plt.title('Loss Curves Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Loss (RMSE)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# save plot as image
plt.savefig('loss_curves.png', dpi=300)

/tmp/ipykernel_463934/1225307238.py:14: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Hyperparameter Search (Learning Rate, L1 Penalty, L2 Penalty)

In [ ]:
# grid search for best learning rate - on normal Linear Regression
from pages.B_Train_Model import LinearRegression

learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.5]
best_rmse = float('inf')
best_lr = None
best_model = None

for lr in learning_rates:
    model = LinearRegression(learning_rate=lr, num_iterations=200)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    current_rmse = rmse(y_val, y_val_pred)

    print(f"Learning Rate: {lr}, RMSE: {current_rmse:.4f}")

    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_lr = lr
        best_model = model

print(f"\nBest Learning Rate: {best_lr}, RMSE: {best_rmse:.4f}")

2025-05-08 14:20:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.0001, RMSE: 0.5603


2025-05-08 14:20:09.568 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.571 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.572 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.577 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.001, RMSE: 0.4926


2025-05-08 14:20:09.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.901 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.01, RMSE: 0.3000


2025-05-08 14:20:10.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.1, RMSE: 0.2973


2025-05-08 14:20:10.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.5, RMSE: 0.2987

Best Learning Rate: 0.1, RMSE: 0.2973


In [ ]:
# grid search to choose l1 and l2 penalty
from pages.B_Train_Model import LassoRegression

l1_penalties = [0.01, 0.1, 0.5, 1.0]
best_rmse = float('inf')
best_l1 = None
best_lasso_model = None

for l1 in l1_penalties:
    model = LassoRegression(learning_rate=best_lr, num_iterations=200, l1_penalty=l1)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    current_rmse = rmse(y_val, y_val_pred)

    print(f"L1 Penalty: {l1}, RMSE: {current_rmse:.4f}")

    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_l1 = l1
        best_lasso_model = model

print(f"\nBest L1 Penalty: {best_l1}, RMSE: {best_rmse:.4f}")

2025-05-08 14:20:06.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.304 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.304 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.304 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.307 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.307 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.307 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.308 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 0.01, RMSE: 0.2973


2025-05-08 14:20:06.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 0.1, RMSE: 0.2973


2025-05-08 14:20:07.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 0.5, RMSE: 0.2972


2025-05-08 14:20:07.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.436 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 1.0, RMSE: 0.2972

Best L1 Penalty: 1.0, RMSE: 0.2972


In [ ]:
from pages.B_Train_Model import RidgeRegression

l2_penalties = [0.01, 0.1, 0.5, 1.0]
best_rmse = float('inf')
best_l2 = None
best_ridge_model = None

for l2 in l2_penalties:
    model = RidgeRegression(learning_rate=best_lr, num_iterations=200, l2_penalty=l2)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    current_rmse = rmse(y_val, y_val_pred)

    print(f"L2 Penalty: {l2}, RMSE: {current_rmse:.4f}")

    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_l2 = l2
        best_ridge_model = model

print(f"\nBest L2 Penalty: {best_l2}, RMSE: {best_rmse:.4f}")

2025-05-08 14:20:07.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 0.01, RMSE: 0.2973


2025-05-08 14:20:08.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 0.1, RMSE: 0.2973


2025-05-08 14:20:08.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 0.5, RMSE: 0.2973


2025-05-08 14:20:08.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 1.0, RMSE: 0.2973

Best L2 Penalty: 0.01, RMSE: 0.2973
